# PyCity Schools Analysis

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# District Summary

In [122]:
#Calculate total number of schools in district
total_schools = school_data_complete['school_name'].nunique()

#Calculate total number of students in district
total_students = school_data_complete['student_name'].count()

#Calculate total budget in district
unique_budgets = school_data_complete.drop_duplicates(['school_name', 'budget']).groupby('school_name').agg({'budget':'sum'})
total_budget = unique_budgets['budget'].sum()

#Calculate average math score
average_math_score = school_data_complete['math_score'].mean()

#Calculate average reading score
average_reading_score = school_data_complete['reading_score'].mean()

#Calculate overall average score
overall_average_score = (average_math_score + passing_reading_score)/2

#Create DataFrame of students with a passing math score
passed_math = len(school_data_complete[(school_data_complete['math_score'] >= 70)]) 

#Calculate percentage of students with a passing math score
passing_math_score = (passed_math / total_students) * 100

#Create DataFrame of students with a passing reading score
passed_reading = len(school_data_complete[(school_data_complete['reading_score'] >= 70)]) 

#Calculate percentage of students with a passing reading score
passing_reading_score = (passed_reading / total_students) * 100

#Create new data frame for District Summary
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                 "Total Students": [total_students],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [average_math_score],
                                 "Average Reading Score": [average_reading_score],
                                 "Overall Average Score": [overall_average_score],
                                 "% Passing Math": [passing_math_score],
                                 "% Passing Reading": [passing_reading_score]
                                })
#Format columns to more readable format
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

#Display District Summary data frame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428.00",78.985371,81.87784,82.395417,74.980853,85.805463


# School Summary

In [172]:
#Create new list with school name and type
school_frame_long = school_data_complete[["school_name", "type"]]

#Parse list to one entry per school
school_summary_raw = school_frame_long.drop_duplicates(subset=None, keep='first', inplace=False)

#Alphabetize data frame by school
school_summary = school_summary_raw.sort_values(by=['school_name'])

#Convert column to index
school_summary.set_index('school_name', inplace=True)

#Determine total students in each school
school_students_raw = school_data_complete.school_name.value_counts()

#Copy original data frame to new one
school_frame_long = school_data_complete

#Count number of students in each school
school_students = pd.DataFrame(school_frame_long.school_name.value_counts())

#Rename "school_name" to more accurate title
school_students2 = school_students.rename(columns={'school_name': 'Total Students'})

#Rename index title
school_students2.index.name = 'school_name'

#Merge number of students with school summary
school_summary2 = pd.merge(school_summary, school_students2[['Total Students']],on='school_name', how='left')

#Create data frame of budgets for each school
school_budget = school_data_complete[["school_name", "budget"]]

#Convert column to index
school_budget.set_index('school_name', inplace=True)

#Merge budget with school summary
school_summary3 = pd.merge(school_summary2, school_budget2[['budget']], on='school_name', how='left')

#Parse list to one entry per school
school_summary4 = school_summary3.drop_duplicates(subset=None, keep='first', inplace=False)

# Divide total budget by total students to get buget per student and add to DataFrame
school_summary4['Per Student Budget'] = school_summary4['budget'] / school_summary4['Total Students']

# Calculate averages of each column for each school
school_averages = school_data_complete.groupby('school_name').mean()

#Merge math score with school summary
school_summary5 = pd.merge(school_summary4, school_averages[['math_score']], on='school_name', how='left')

#Merge reading score with school summary
school_summary6 = pd.merge(school_summary5, school_averages[['reading_score']], on='school_name', how='left')

#Create list of number of students with a passing math score in each school
school_passed_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['math_score'].count()

#Convert list to data frame
school_passed_math2 = pd.DataFrame({'passing_students':school_passed_math})

# Merge data frame of students with a passing math score in each school with data frame of total students in each school
school_passed_math3 = pd.merge(school_passed_math2, school_students2[['Total Students']], on='school_name', how='left')

# Calculate percentage of students with a passing math score in each school
school_passed_math4 = (school_passed_math3['passing_students'] / school_passed_math3['Total Students']) * 100

#Convert list to data frame
school_passed_math5 = pd.DataFrame({'passing_math_percentage':school_passed_math4})

# Merge data frame of percentage of students with a passing math score in each school with school summary
school_summary7 = pd.merge(school_summary6, school_passed_math5[['passing_math_percentage']], on='school_name', how='left')

#Create list of number of students with a passing reading score in each school
school_passed_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['reading_score'].count()

#Convert list to data frame
school_passed_reading2 = pd.DataFrame({'passing_students':school_passed_reading})

# Merge data frame of students with a passing reading score in each school with data frame of total students in each school
school_passed_reading3 = pd.merge(school_passed_reading2, school_students2[['Total Students']], on='school_name', how='left')

# Calculate percentage of students with a passing reading score in each school
school_passed_reading4 = (school_passed_reading3['passing_students'] / school_passed_reading3['Total Students']) * 100

#Convert list to data frame
school_passed_reading5 = pd.DataFrame({'passing_reading_percentage':school_passed_reading4})

# Merge data frame of percentage of students with a passing reading score in each school with school summary
school_summary8 = pd.merge(school_summary7, school_passed_reading5[['passing_reading_percentage']], on='school_name', how='left')

#Calculate overall passing rate for each school and add to school summary
school_summary8['% Overall Passing Rate'] = (school_summary8['passing_math_percentage'] + school_summary8['passing_reading_percentage'])/2

# Rename column names
school_summary8.columns = ['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']

# Delete index column name
del school_summary8.index.name

#Format columns to more readable format
school_summary8["Total School Budget"] = school_summary8["Total School Budget"].map("${:,.2f}".format)
school_summary8["Per Student Budget"] = school_summary8["Per Student Budget"].map("${:,.2f}".format)

#Display school summary
school_summary8

C:\Users\theju\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


# Top Performing Schools (By Passing Rate)

In [173]:
#Sort school summary by descending overall passing rate
top_schools = school_summary8.sort_values(by=['% Overall Passing Rate'], ascending=False)

#Display top 5 schools by overall passing rate
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Passing Rate)

In [174]:
#Sort school summary by ascending overall passing rate
bottom_schools = school_summary8.sort_values(by=['% Overall Passing Rate'])

#Display bottom 5 schools by overall passing rate
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


# Math Scores by Grade

In [151]:
#Only pull student data for 9th graders
grade_9 = school_data_complete.loc[school_data_complete['grade'] == '9th']

# Calculate averages of each column for each school
grade_9_average = grade_9.groupby('school_name').mean()

#Remove extra columns from 9th grade data frame, leaving school name, reading score, and math score
grade_9_average2 = grade_9_average[["reading_score", "math_score"]]

#Only pull student data for 10th graders
grade_10 = school_data_complete.loc[school_data_complete['grade'] == '10th']

# Calculate averages of each column for each school
grade_10_average = grade_10.groupby('school_name').mean()

#Remove extra columns from 10th grade data frame, leaving school name, reading score, and math score
grade_10_average2 = grade_10_average[["reading_score", "math_score"]]

#Only pull student data for 11th graders
grade_11 = school_data_complete.loc[school_data_complete['grade'] == '11th']

# Calculate averages of each column for each school
grade_11_average = grade_11.groupby('school_name').mean()

#Remove extra columns from 11th grade data frame, leaving school name, reading score, and math score
grade_11_average2 = grade_11_average[["reading_score", "math_score"]]

#Only pull student data for 12th graders
grade_12 = school_data_complete.loc[school_data_complete['grade'] == '12th']

# Calculate averages of each column for each school
grade_12_average = grade_12.groupby('school_name').mean()

#Remove extra columns from 12th grade data frame, leaving school name, reading score, and math score
grade_12_average2 = grade_12_average[["reading_score", "math_score"]]

#Merge 9th and 10th grade data frames
grade_9_10 = pd.merge(grade_9_average2, grade_10_average2, on='school_name', how='left')

#Merge 9th/10th and 11th grade data frames
grade_9_10_11 = pd.merge(grade_9_10, grade_11_average2, on='school_name', how='left')

#Merge 9th/10th/11th and 12th grade data frames
math_scores_by_grade = pd.merge(grade_9_10_11, grade_12_average2, on='school_name', how='left')

#Remove extra columns from combined data frame, leaving only math scores
del math_scores_by_grade['reading_score_x']
del math_scores_by_grade['reading_score_y']

# Rename column names
math_scores_by_grade.columns = ['9th', '10th', '11th', '12th']

# Delete index column name
del math_scores_by_grade.index.name

math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores By Grade

In [152]:
#Merge 9th/10th/11th and 12th grade data frames
reading_scores_by_grade = pd.merge(grade_9_10_11, grade_12_average2, on='school_name', how='left')

#Remove extra columns from combined data frame, leaving only reading scores
del reading_scores_by_grade['math_score_x']
del reading_scores_by_grade['math_score_y']

# Rename column names
reading_scores_by_grade.columns = ['9th', '10th', '11th', '12th']

# Delete index column name
del reading_scores_by_grade.index.name

reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [216]:
# Create new dataframe with only desired columns
school_spending = school_summary8[['Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Set index title
school_spending.index.name = 'school_name'

# Remove formatting of Per Student Budget column
school_spending['Per Student Budget'] = school_spending['Per Student Budget'].str.replace('$', '')
school_spending['Per Student Budget'] = school_spending['Per Student Budget'].map(float)
school_spending['Per Student Budget'] = school_spending['Per Student Budget'].astype(int)

# Establish spending bins and group names
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#Slice the data and place it into bins
school_spending2 = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names)

#Convert series to data frame
school_spending3 = school_spending2.to_frame(name='bins')

#Merge bins data frame with school summary
school_spending4 = pd.merge(school_summary8, school_spending3, on='school_name', how='left')

#Create a group by object based upon bins
school_spending5 = school_spending4.groupby("bins")

#Get the average of each column
school_spending6 = school_spending5[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

# Set index title
school_spending6.index.name = 'Spending Ranges (Per Student)'

# Display scores by school spending
school_spending6

C:\Users\theju\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\theju\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\theju\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


# Scores by School Size

In [223]:
# Create new dataframe with only desired columns
school_size = school_summary8[['Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Set index title
school_size.index.name = 'school_name'

# Establish size bins and group names
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Slice the data and place it into bins
school_size2 = pd.cut(school_size["Total Students"], size_bins, labels=group_names)

#Convert series to data frame
school_size3 = school_size2.to_frame(name='bins')

#Merge bins data frame with school summary
school_size4 = pd.merge(school_summary8, school_size3, on='school_name', how='left')

#Create a group by object based upon bins
school_size5 = school_size4.groupby("bins")

#Get the average of each column
school_size6 = school_size5[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

# Set index title
school_size6.index.name = 'School Size'

# Display scores by school size
school_size6

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


# Scores by School Type

In [226]:
# Create new dataframe with only desired columns
school_type = school_summary8[['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# Set index title
school_type.index.name = 'school_name'

#Create a group by object based upon bins
school_type2 = school_type.groupby("School Type")

#Get the average of each column
school_type3 = school_type2[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].mean()

# Set index title
school_type3.index.name = 'School Type'

# Display scores by school type
school_type3

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


# Findings

There are a few easily noticeable trends using the data. Firstly, charter schools have significantly higher passing rates versus district schools. Secondly, students from large schools are quite less likely to pass than those from small or medium-sized schools. Finally, and most surprisingly, it seems that the less money spent on a student, the more likely they are to succeed. One would expect the opposite relationship.